In [51]:
import pandas as pd
import datetime

In [3]:
path = '/Users/ziyuanhan/Desktop/ml2/final_project'

In [4]:
attribute = '/attributes.csv'
events = '/events.csv'
messages = '/messages.csv'
sessions = '/sessions.csv'

In [5]:
df_attribute = pd.read_csv(path+attribute, error_bad_lines=False, warn_bad_lines=False)
df_attribute.sample(frac=0.017, replace=False, random_state=1).to_csv(path+"/attribute_sample.csv", index=False)

In [6]:
df_events = pd.read_csv(path+events, error_bad_lines=False, warn_bad_lines=False)
df_events.sample(frac=0.036, replace=False, random_state=1).to_csv(path+"/events_sample.csv", index=False)

In [7]:
df_messages = pd.read_csv(path+messages)

In [8]:
df_sessions = pd.read_csv(path+sessions)
df_sessions.sample(frac=0.3, replace=False, random_state=1).to_csv(path+"/sessions_sample.csv", index=False)

In [13]:
df_attribute.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184880449 entries, 0 to 184880448
Data columns (total 5 columns):
app_id             int64
session_id         int64
attribute          int64
attribute_value    object
user_id_hash       object
dtypes: int64(3), object(2)
memory usage: 36.3 GB


In [11]:
df_events.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 6 columns):
app_id             int64
session_id         int64
event              object
event_timestamp    int64
event_value        float64
user_id_hash       object
dtypes: float64(1), int64(3), object(2)
memory usage: 22.3 GB


In [18]:
df_attribute['app_id'].nunique()

1

In [14]:
df_events.head(5)

,app_id,session_id,event,event_timestamp,event_value,user_id_hash
0,4724682771660800,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,4724682771660800,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,4724682771660800,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,4724682771660800,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4724682771660800,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [17]:
df_events['app_id'].nunique()

1

In [19]:
df_events['session_id'].nunique()

5009421

In [24]:
df_events['user_id_hash'].nunique()

621001

In [45]:
df_events['event_value'].describe()

count    1.119465e+08
mean     1.476199e+04
std      7.948289e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.294967e+09
Name: event_value, dtype: float64

In [20]:
df_after = df_events.drop(['app_id'],axis=1)

In [21]:
df_after.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 5 columns):
session_id         int64
event              object
event_timestamp    int64
event_value        float64
user_id_hash       object
dtypes: float64(1), int64(2), object(2)
memory usage: 21.4 GB


In [25]:
# here is a handy function from fast.ai
def proc_col(col):
    """Encodes a pandas column with continous ids. 
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [49]:
movie2id, movie_col, num_movies = proc_col(df_after["user_id_hash"])

In [50]:
with open("../processed/user_hash_dic", 'w') as f:
    for key, value in movie2id.items():
        f.write('%s:%s\n' % (key, value))

In [29]:
def encode_data(df):
    
    ### BEGIN SOLUTION
    user2id, user_col, num_users = proc_col(df["session_id"])
    df["sessionId"] = user_col
    movie2id, movie_col, num_movies = proc_col(df["user_id_hash"])
    df["userhashId"] = movie_col
    ### END SOLUTION
    return df, num_users, num_movies

In [32]:
df_tmp,_,_ = encode_data(df_after)

In [33]:
df_tmp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 7 columns):
session_id         int64
event              object
event_timestamp    int64
event_value        float64
user_id_hash       object
sessionId          int64
userhashId         int64
dtypes: float64(1), int64(4), object(2)
memory usage: 23.1 GB


In [37]:
df_tmp.head()

,session_id,event,event_timestamp,event_value,user_id_hash,sessionId,userhashId
0,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0
1,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0
2,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,0
3,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2,0
4,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,2,0


In [38]:
df_tmp = df_tmp.drop(['session_id','user_id_hash'], axis=1)
df_tmp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 5 columns):
event              object
event_timestamp    int64
event_value        float64
sessionId          int64
userhashId         int64
dtypes: float64(1), int64(3), object(1)
memory usage: 9.7 GB


In [39]:
df_tmp.head()

,event,event_timestamp,event_value,sessionId,userhashId
0,45,1542215397132,0.0,0,0
1,45,1542215484895,0.0,0,0
2,.m5100869650219008,1541124410372,0.0,1,0
3,4,1543713091129,0.0,2,0
4,6,1543713093116,0.0,2,0


In [41]:
df_tmp['sessionId'] = df_tmp['sessionId'].astype('uint32')

In [42]:
df_tmp['userhashId'] = df_tmp['userhashId'].astype('uint32')

In [46]:
df_tmp['event_value'] = df_tmp['event_value'].astype('float32')

In [47]:
df_tmp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111946597 entries, 0 to 111946596
Data columns (total 5 columns):
event              object
event_timestamp    int64
event_value        float32
sessionId          uint32
userhashId         uint32
dtypes: float32(1), int64(1), object(1), uint32(2)
memory usage: 8.4 GB


In [52]:
time = df_tmp['event_timestamp'][0]

In [53]:
time

1542215397132

In [54]:
datetime.datetime.fromtimestamp(time/1000.0)

datetime.datetime(2018, 11, 14, 9, 9, 57, 132000)

In [56]:
datetime.datetime.utcfromtimestamp(time//1000).replace(microsecond=time%1000*1000)

datetime.datetime(2018, 11, 14, 17, 9, 57, 132000)

In [57]:
time = [datetime.datetime.utcfromtimestamp(i//1000).
        replace(microsecond=i%1000*1000) for i in df_tmp['event_timestamp']]

In [58]:
min(time)

datetime.datetime(2018, 10, 1, 7, 0, 4, 526000)

In [59]:
max(time)

datetime.datetime(2018, 12, 14, 23, 59, 59, 977000)

In [62]:
int(df_tmp['event'][2])

ValueError: invalid literal for int() with base 10: '.m5100869650219008'

In [63]:
boo = []
boo.append(1)
boo

[1]

In [70]:
boo = []
for i in df_tmp['event']:
    try:
        int(i)
        boo.append(True)
    except ValueError:
        boo.append(False)
len(boo) 

111946597

In [72]:
df_tmp_no_message = df_tmp[boo]

In [73]:
df_tmp_no_message.head()

,event,event_timestamp,event_value,sessionId,userhashId
0,45,1542215397132,0.0,0,0
1,45,1542215484895,0.0,0,0
3,4,1543713091129,0.0,2,0
4,6,1543713093116,0.0,2,0
5,5,1543713101700,1.0,2,0


In [74]:
len(df_tmp_no_message)

109627798

In [68]:
len(df_tmp)

111946597

In [71]:
boo[0:3]

[True, True, False]